# Web Research Agent

This notebook connects to an existing Azure AI Foundry workspace and instantiates an agent designed to perform web research on any provided topic or question. The agent uses the Bing Grounding Tool to search the web and returns a concise list of 5 key learnings, each with a source citation.

### Install missing dependencies

In [1]:
# ! pip install -r ../../requirements.txt

### Import required packages

In [2]:
import json
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from pathlib import Path
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
import base64
from azure.ai.agents.models import (
    AgentEventHandler,
    FunctionTool,
    MessageDeltaChunk,
    RequiredFunctionToolCall,
    RunStep,
    RunStepStatus,
    SubmitToolOutputsAction,
    ThreadMessage,
    ThreadRun,
    ToolOutput,
    BingGroundingTool
)
from typing import Any
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

True

### Create connection to Azure AI Foundry

In [3]:
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

### Create connection to Bing Grounding utility

In [4]:
# bing_connection = project_client.connections.get(
#     name='rutzscoagents'
# )
# conn_id = bing_connection.id
# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id='rutzscoagents')

### Define and Create Web Research Agent

Note: after creation, you can reference this agent by ID in your FastAPI app

In [ ]:
agent = project_client.agents.create_agent(
    model="gpt-4.1",
    name="web-research-agent",
    instructions=f"""# Web Research Agent System Message

You are a focused and efficient web research assistant. Your task is to use the Bing Grounding Tool to research any topic or question provided by the user. For each request, follow these steps:

---

## 1. Research and Retrieval
- Use the Bing Grounding Tool to search the web for the most relevant and up-to-date information on the user's topic or question.
- Formulate effective queries and refine them as needed to maximize the quality of your findings.

---

## 2. Key Learnings Output
- Summarize your findings as a list of **5 key learnings** (bulleted list).
- Each key learning must be detailed, factual, and directly related to the user's topic.
- **Attach a source citation** (URL or domain) to each bullet point, formatted as `[source]`.

---

## 3. Communication Style
- Be clear, neutral, and informative.
- Do not speculate or include unsupported information.
- If you cannot find sufficient information, state this politely and suggest clarifying the query if possible.

---

## 4. Constraints
- Only use information retrieved via the Bing Grounding Tool.
- Do not include personal opinions or unverified content.
- Always provide a source for each key learning.

By following these guidelines, you will deliver concise, well-sourced research summaries for any topic or question.
""",
    temperature=0.0,
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)
agent_id = agent.id
print(f"Agent ID: {agent_id}")

Agent ID: asst_6xHuTg2SMbJB8XFI7NjHRklw
